In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
con = mysql.connector.connect(user='ga1009', 
                        host='orion.bio.nyu.edu', database='godb', password='mkatari@nyu')

In [ ]:
c = con.cursor()

In [ ]:

c.execute("show tables")
godb_list = []
for table in c:
     godb_list.append(table[0])

In [ ]:
godb_list

['association', 'gene_product', 'graph_path', 'species', 'term']

In [ ]:
pd.set_option('display.max_columns', 300000)
pd.set_option('display.max_rows', 300000)

In [ ]:
pd.read_sql("""
        select * 
        from term 
        where is_obsolete=0
        limit 5
""", con)

,id,name,term_type,acc,is_obsolete,is_root,is_relation
0,1,is_a,relationship,is_a,0,0,1
1,2,consider,metadata,consider,0,0,1
2,3,replaced_by,metadata,replaced_by,0,0,1
3,4,Grouping classes that can be excluded,subset,goantislim_grouping,0,0,0
4,5,Term not to be used for direct annotation,subset,gocheck_do_not_annotate,0,0,0


In [ ]:
pd.read_sql("""
        select * 
        from gene_product where species_id =\"462613\"
        limit 5
""", con)

,id,symbol,dbxref_id,species_id,type_id,full_name
0,271693,PDCD6,424402,462613,46038,"HCG1985580, isoform CRA_c"
1,271694,CDK1,424403,462613,46038,Cyclin-dependent kinase 1
2,271695,SERPINB6,424405,462613,46038,Serpin B6
3,271696,APOC2,424406,462613,46038,Apolipoprotein C-II isoform 1
4,271697,TADA2L,424407,462613,46038,Transcriptional adapter 2-alpha


In [ ]:
pd.read_sql("""
        select * 
        from association limit 5
""", con)

,id,term_id,gene_product_id,is_not,role_group,assocdate,source_db_id
0,1,311,1,0,None,20161225,11
1,2,2892,1,0,None,20161225,11
2,3,6679,1,0,None,20161225,11
3,4,2892,2,0,None,20161225,11
4,5,6679,2,0,None,20161225,11


In [ ]:
pd.read_sql("""
        select * 
        from graph_path limit 5
""", con)

,id,term1_id,term2_id,relationship_type_id,distance,relation_distance
0,1,28,24,1,1,1
1,2,28,27,1,1,1
2,3,31,16028,26,1,1
3,4,31,17901,1,1,1
4,5,32,2455,26,1,1


In [ ]:
pd.read_sql("""
        select * 
        from species limit 5
""", con)

,id,ncbi_taxa_id,common_name,lineage_string,genus,species,parent_id,left_value,right_value,taxonomic_rank
0,1,127219,None,None,Tellervini,,504786,1047896,1047901,None
1,2,695965,None,None,Halococcus,sp. KeC-02,279841,1805437,1805438,None
2,3,1860139,None,None,Chitinispirillum,,977523,1946747,1946750,None
3,4,564321,None,None,Rhodococcus,sp. 302BRRJ,626198,2103272,2103273,None
4,5,544357,None,None,Araotes,,1040837,1041141,1041146,None


In [ ]:
result1 = pd.read_sql(""" select * from species where genus=\"Homo\" and species=\"sapiens\" """, con)
result1

,id,ncbi_taxa_id,common_name,lineage_string,genus,species,parent_id,left_value,right_value,taxonomic_rank
0,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None


In [ ]:
result2 = pd.read_sql(""" select * from association a join term b join gene_product c join species d
                        on b.id=a.term_id and c.id=a.gene_product_id and c.species_id=d.id
                        where (d.genus=\"Homo\" and d.species=\"sapiens\") and b.is_obsolete=0
                        
""", con)
result2

,id,term_id,gene_product_id,is_not,role_group,assocdate,source_db_id,id,name,term_type,acc,is_obsolete,is_root,is_relation,id,symbol,dbxref_id,species_id,type_id,full_name,id,ncbi_taxa_id,common_name,lineage_string,genus,species,parent_id,left_value,right_value,taxonomic_rank
0,1368522,1039,271693,0,None,20160507,121,1039,calcium ion binding,molecular_function,GO:0005509,0,0,0,271693,PDCD6,424402,462613,46038,"HCG1985580, isoform CRA_c",462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
1,7852135,1039,271693,0,None,20160507,121,1039,calcium ion binding,molecular_function,GO:0005509,0,0,0,271693,PDCD6,424402,462613,46038,"HCG1985580, isoform CRA_c",462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
2,1368523,3860,271694,0,None,20160507,307,3860,protein serine/threonine kinase activity,molecular_function,GO:0004674,0,0,0,271694,CDK1,424403,462613,46038,Cyclin-dependent kinase 1,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
3,1368524,4563,271694,0,None,20160507,307,4563,ATP binding,molecular_function,GO:0005524,0,0,0,271694,CDK1,424403,462613,46038,Cyclin-dependent kinase 1,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
4,1368525,4668,271694,0,None,20101115,114,4668,nucleus,cellular_component,GO:0005634,0,0,0,271694,CDK1,424403,462613,46038,Cyclin-dependent kinase 1,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710411,254012528,3014,42971362,0,None,20160507,121,3014,membrane,cellular_component,GO:0016020,0,0,0,42971362,HLA-DRB1,43204029,462613,46038,MHC class II antigen,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
2710412,254012529,343,42971362,0,None,20160507,307,343,integral component of membrane,cellular_component,GO:0016021,0,0,0,42971362,HLA-DRB1,43204029,462613,46038,MHC class II antigen,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
2710413,254012530,13309,42971362,0,None,20160507,121,13309,antigen processing and presentation,biological_process,GO:0019882,0,0,0,42971362,HLA-DRB1,43204029,462613,46038,MHC class II antigen,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None
2710414,254012531,3056,42971362,0,None,20160507,121,3056,MHC class II protein complex,cellular_component,GO:0042613,0,0,0,42971362,HLA-DRB1,43204029,462613,46038,MHC class II antigen,462613,9606,human,None,Homo,sapiens,896781,1683028,1683033,None


In [ ]:
result1=pd.read_sql(""" select distinct(b.acc), c.symbol from association a join term b join gene_product c join species d
                        on b.id=a.term_id and c.id=a.gene_product_id and c.species_id=d.id
                        where (d.genus=\"Homo\" and d.species=\"sapiens\") and b.is_obsolete=0
""", con)

result1.to_csv(r'./result1.csv', sep='\t', index=False)

In [ ]:
common_name = "\"human\""
tmp_query = """select distinct(b.acc), c.symbol from association a join term b 
                join gene_product c join species d
                on b.id=a.term_id and c.id=a.gene_product_id and c.species_id=d.id
                where d.common_name=%s and b.is_obsolete=0
"""%(common_name)

In [ ]:
pd.read_sql(tmp_query, con)

,acc,symbol
0,GO:0005509,PDCD6
1,GO:0004674,CDK1
2,GO:0005524,CDK1
3,GO:0005634,CDK1
4,GO:0005737,CDK1
...,...,...
1216178,GO:0042612,HLA A
1216179,GO:0002474,HLA C
1216180,GO:0006955,HLA C
1216181,GO:0042605,HLA C


In [ ]:
z = pd.read_sql("""select * from term a join graph_path b on a.id=b.relationship_type_id limit 20

""", con)

z.dtypes

id                       int64
name                    object
term_type               object
acc                     object
is_obsolete              int64
is_root                  int64
is_relation              int64
id                       int64
term1_id                 int64
term2_id                 int64
relationship_type_id     int64
distance                 int64
relation_distance        int64
dtype: object

In [ ]:
import os
os.getcwd()

'C:\\Users\\tanub\\Documents\\BIOL GA 1009\\Home_Work\\Homework 3'

In [ ]:
relationship=pd.read_sql("""select * from term 
""", con)
relationship.to_csv(r'./relationship.csv', index=False)

In [ ]:
graph_path=pd.read_sql("""select * from graph_path 
""", con)
graph_path.to_csv(r'./graph_path.csv', index=False)

In [ ]:
species=pd.read_sql("""
            select * from species
            
""", con)
species.to_csv(r'./species.csv', index=False )

In [ ]:
gene_product=pd.read_sql("""
            select * from gene_product 
            limit 100
""", con)
gene_product.to_csv(r'./gene_product.csv', index=False)

In [ ]:
association=pd.read_sql("""
            select * from association
            limit 100
""", con)
association.to_csv(r'./association.csv', index=False)

In [ ]:
pd.read_sql("""
            select * from graph_path limit 5
""", con)

,id,term1_id,term2_id,relationship_type_id,distance,relation_distance
0,1,28,24,1,1,1
1,2,28,27,1,1,1
2,3,31,16028,26,1,1
3,4,31,17901,1,1,1
4,5,32,2455,26,1,1


In [ ]:
pd.read_sql("""
            select * from term a join graph_path b 
            on a.id=b.relationship_type_id 
            where a.term_type = \"relationship\"
            
            limit 10
""", con)

,id,name,term_type,acc,is_obsolete,is_root,is_relation,id,term1_id,term2_id,relationship_type_id,distance,relation_distance
0,1,is_a,relationship,is_a,0,0,1,1,28,24,1,1,1
1,1,is_a,relationship,is_a,0,0,1,2,28,27,1,1,1
2,1,is_a,relationship,is_a,0,0,1,4,31,17901,1,1,1
3,1,is_a,relationship,is_a,0,0,1,7,32,13365,1,1,1
4,1,is_a,relationship,is_a,0,0,1,8,32,13366,1,1,1
5,1,is_a,relationship,is_a,0,0,1,10,32,16471,1,1,1
6,1,is_a,relationship,is_a,0,0,1,11,32,16472,1,1,1
7,1,is_a,relationship,is_a,0,0,1,13,35,34,1,1,1
8,1,is_a,relationship,is_a,0,0,1,14,35,41,1,1,1
9,1,is_a,relationship,is_a,0,0,1,15,35,75,1,1,1


In [ ]:
pd.read_sql("""select distinct(c.parent), d.child from
            (select a.acc as parent from term a join graph_path b
            on a.id=b.term1_id where (a.term_type=\"biological_process\" or a.term_type=\"molecular_function\" 
            or a.term_type=\"cellular_component\")) as c inner join
            (select b.acc as child from term b join graph_path a
            on b.id=a.term2_id where (b.term_type=\"biological_process\" or b.term_type=\"molecular_function\" 
            or b.term_type=\"cellular_component\")) as d
            limit 30
""", con)

,parent,child
0,GO:0000001,GO:0000001
1,GO:0000002,GO:0000001
2,GO:0000003,GO:0000001
3,GO:0000005,GO:0000001
4,obsolete_molecular_function,GO:0000001
5,GO:0042254,GO:0000001
6,GO:0044183,GO:0000001
7,GO:0051082,GO:0000001
8,GO:0000006,GO:0000001
9,GO:0000007,GO:0000001


In [ ]:
pd.read_sql("""select f.ACC from
                (select a.id as ID, b.term1_id, b.term2_id
                from term a inner join graph_path b
                on a.id=b.relationship_type_id
                where a.term_type=\"relationship\" and a.id=1) as e
                inner join
                (select c.id, c.acc as ACC, d.term1_id from
                term c inner join graph_path d
                on c.id=d.term1_id) as f
                limit 10
""", con)

,ACC
0,goantislim_grouping
1,goantislim_grouping
2,goantislim_grouping
3,goantislim_grouping
4,goantislim_grouping
5,goantislim_grouping
6,goantislim_grouping
7,goantislim_grouping
8,goantislim_grouping
9,goantislim_grouping


In [ ]:
pd.read_sql("""select t1.acc, t2.acc from
                (select a.id as id, a.acc as acc from term a 
                join graph_path b on
                a.id=b.term1_id where a.acc like "%GO:%") as t1
                inner join
                (select a.id as id, a.acc as acc from term a
                join graph_path b on
                a.id=b.term2_id where a.acc like "%GO:%") as t2
                inner join
                (select a.id as id, a.acc as acc from term a 
                join graph_path b on
                a.id=b.relationship_type_id where 
                a.term_type=\"relationship\") as t3
                limit 5
""", con)

,acc,acc
0,GO:0000001,GO:0000001
1,GO:0000001,GO:0000001
2,GO:0000001,GO:0000001
3,GO:0000001,GO:0000001
4,GO:0000001,GO:0000001


In [ ]:
result3=pd.read_sql("""select t1.acc as Parent, t2.acc as Child from graph_path a 
                join term t1 on a.term1_id=t1.id 
                join term t2 on a.term2_id=t2.id
                where t1.acc like "%GO%" and
                t2.acc like "%GO%" and a.relationship_type_id=1
                and a.term1_id!=a.term2_id
                
""", con)
result3.to_csv(r'./result3.csv', sep='\t', index=False)

In [ ]:
pd.read_sql("""select distinct(a.id), a.acc as acc from term a 
                join graph_path b on
                a.id=b.term2_id where a.acc like "%GO:%"
                and b.relationship_type_id=1
                limit 10
""", con)

,id,acc
0,29,GO:0000001
1,31,GO:0000002
2,32,GO:0000003
3,34,GO:0000005
4,36,GO:0042254
5,37,GO:0044183
6,38,GO:0051082
7,39,GO:0000006
8,40,GO:0000007
9,41,GO:0000008


In [ ]:
pd.read_sql("""select * from term a 
                join graph_path b
                on a.id=b.relationship_type_id
                where b.relationship_type_id=1
""", con)

,id,name,term_type,acc,is_obsolete,is_root,is_relation,id,term1_id,term2_id,relationship_type_id,distance,relation_distance
0,1,is_a,relationship,is_a,0,0,1,1,28,24,1,1,1
1,1,is_a,relationship,is_a,0,0,1,2,28,27,1,1,1
2,1,is_a,relationship,is_a,0,0,1,4,31,17901,1,1,1
3,1,is_a,relationship,is_a,0,0,1,7,32,13365,1,1,1
4,1,is_a,relationship,is_a,0,0,1,8,32,13366,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
682012,1,is_a,relationship,is_a,0,0,1,1213432,46028,46028,1,0,0
682013,1,is_a,relationship,is_a,0,0,1,1213433,46029,46029,1,0,0
682014,1,is_a,relationship,is_a,0,0,1,1213434,46030,46030,1,0,0
682015,1,is_a,relationship,is_a,0,0,1,1213435,46031,46031,1,0,0


In [ ]:
pd.read_sql("""select t1.acc as Parent, t2.acc as Child from graph_path a 
                join term t1 on a.term1_id=t1.id 
                join term t2 on a.term2_id=t2.id
                where t1.acc like "%GO%" and
                t2.acc like "%GO%" and a.relationship_type_id=1
                and a.term1_id!=a.term2_id
                
""", con)

,Parent,Child
0,GO:0000002,GO:0033955
1,GO:0000003,GO:0019953
2,GO:0000003,GO:0019954
3,GO:0000003,GO:0032504
4,GO:0000003,GO:0032505
...,...,...
587970,GO:0051704,GO:0075164
587971,GO:0051704,GO:0075166
587972,GO:0051704,GO:0075167
587973,GO:0008150,GO:1904956


In [ ]:
pd.read_sql("""select t1.acc as Parent, t2.acc as Child from graph_path a 
                join term t1 on a.term1_id=t1.id 
                join term t2 on a.term2_id=t2.id
                where t1.acc like "%GO%" and
                t2.acc like "%GO%" and a.relationship_type_id=1
                
                
""", con)

,Parent,Child
0,GO:0000002,GO:0033955
1,GO:0000003,GO:0019953
2,GO:0000003,GO:0019954
3,GO:0000003,GO:0032504
4,GO:0000003,GO:0032505
...,...,...
633981,GO:2001313,GO:2001313
633982,GO:2001314,GO:2001314
633983,GO:2001315,GO:2001315
633984,GO:2001316,GO:2001316


In [ ]:
pd.read_sql("""select * from graph_path where term2_id=31
                
                
""", con)

,id,term1_id,term2_id,relationship_type_id,distance,relation_distance
0,16318,5944,31,1,1,1
1,116530,5934,31,1,2,2
2,269368,10609,31,1,3,3
3,269369,8423,31,1,4,4
4,269370,33708,31,1,4,4
5,557198,6679,31,1,5,5
6,557199,46032,31,1,6,6
7,1167435,31,31,1,0,0


In [ ]:
con.close()